In [22]:
import pandas as pd
import json
import plotly.express as px

df_position = pd.read_json("data/ais_position.json")
df_static = pd.read_json("data/ais_static.json")
df_position

,timestamp,MMSI,ShipName,lat,lon,COG,NavigationalStatus,RateOfTurn,SOG,Spare,UserID
0,2025-06-18 15:51:25.073315374 +0000 UTC,249986000,SEARUNNER,48.823652,-5.525898,54.5,0,2,11.7,0,249986000
1,2025-06-18 15:51:25.4274521 +0000 UTC,227006680,MISTRAL 10,48.381085,-4.484475,254.0,0,-128,0.0,0,227006680
2,2025-06-18 15:51:26.171333249 +0000 UTC,228267900,F/V FELIR,48.678585,-5.095863,264.7,0,0,3.8,0,228267900
3,2025-06-18 15:51:27.019275073 +0000 UTC,227006650,MISTRAL 7,48.381160,-4.484527,360.0,5,-128,0.0,0,227006650
4,2025-06-18 15:51:27.020139643 +0000 UTC,305961000,ANNA,48.703300,-5.619517,27.8,0,-8,8.8,0,305961000
...,...,...,...,...,...,...,...,...,...,...,...
141,2025-06-18 15:54:43.885797143 +0000 UTC,538010241,DANAE,48.811340,-5.537863,56.8,0,-4,10.8,0,538010241
142,2025-06-18 15:54:45.44243259 +0000 UTC,228796000,FS BELLE POULE,48.280775,-4.586092,206.1,0,-128,4.0,0,228796000
143,2025-06-18 15:54:46.041984828 +0000 UTC,228070700,F/V MELIZENN,48.470912,-5.596402,11.3,7,0,2.8,0,228070700
144,2025-06-18 15:54:46.827426378 +0000 UTC,227592820,TREBERON,48.380217,-4.498227,299.5,15,-128,0.0,0,227592820


In [24]:
# Need to take the latest timestamp from df_static
final_df = pd.merge(df_position, df_static, on='MMSI', how='left')
final_df

,timestamp_x,MMSI,ShipName,lat,lon,COG,NavigationalStatus,RateOfTurn,SOG,Spare,UserID,timestamp_y,Destination,Dimension,Eta,Type
0,2025-06-18 15:51:25.073315374 +0000 UTC,249986000,SEARUNNER,48.823652,-5.525898,54.5,0,2,11.7,0,249986000,NaN,NaN,NaN,NaN,NaN
1,2025-06-18 15:51:25.4274521 +0000 UTC,227006680,MISTRAL 10,48.381085,-4.484475,254.0,0,-128,0.0,0,227006680,NaN,NaN,NaN,NaN,NaN
2,2025-06-18 15:51:26.171333249 +0000 UTC,228267900,F/V FELIR,48.678585,-5.095863,264.7,0,0,3.8,0,228267900,2025-06-18 15:54:34.744256745 +0000 UTC,ROSCOFF,"{'A': 15, 'B': 10, 'C': 4, 'D': 4}","{'Day': 28, 'Hour': 4, 'Minute': 0, 'Month': 1}",30.0
3,2025-06-18 15:51:27.019275073 +0000 UTC,227006650,MISTRAL 7,48.381160,-4.484527,360.0,5,-128,0.0,0,227006650,2025-06-18 15:52:27.175153572 +0000 UTC,BREST,"{'A': 15, 'B': 15, 'C': 3, 'D': 7}","{'Day': 30, 'Hour': 19, 'Minute': 0, 'Month': 3}",52.0
4,2025-06-18 15:51:27.020139643 +0000 UTC,305961000,ANNA,48.703300,-5.619517,27.8,0,-8,8.8,0,305961000,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,2025-06-18 15:54:43.885797143 +0000 UTC,538010241,DANAE,48.811340,-5.537863,56.8,0,-4,10.8,0,538010241,NaN,NaN,NaN,NaN,NaN
142,2025-06-18 15:54:45.44243259 +0000 UTC,228796000,FS BELLE POULE,48.280775,-4.586092,206.1,0,-128,4.0,0,228796000,NaN,NaN,NaN,NaN,NaN
143,2025-06-18 15:54:46.041984828 +0000 UTC,228070700,F/V MELIZENN,48.470912,-5.596402,11.3,7,0,2.8,0,228070700,NaN,NaN,NaN,NaN,NaN
144,2025-06-18 15:54:46.827426378 +0000 UTC,227592820,TREBERON,48.380217,-4.498227,299.5,15,-128,0.0,0,227592820,NaN,NaN,NaN,NaN,NaN


In [25]:
fig = px.scatter_map(final_df, lat="lat", lon="lon", color="NavigationalStatus",
                  color_continuous_scale=px.colors.cyclical.IceFire)

fig.show()